In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Parameters

In [ ]:
# General constants
g = 9.81 # m/s^2
rho = 1025 # kg/m^3
nu = 1.33e-6 # m^2/s

Kv = 1.1 # Velocity coefficient

## Armour Layer

In [11]:
def specific_density(rho_s=2650.0, rho_w=1000.0):
    """
    Calculates the specific density Δ_s.

    Parameters
    ----------
    rho_s : float
        Density of rock material [kg/m³]
    rho_w : float, optional
        Density of water [kg/m³] (default = 1000)

    Returns
    -------
    Delta_s : float
        Specific density [-]
    """
    return (rho_s - rho_w) / rho_w

def dimensionless_particle_diameter(d50, Delta_s, nu, g):
    """
    Calculates the dimensionless particle diameter d_*.

    Parameters
    ----------
    d50 : float
        Median particle diameter [m]
    Delta_s : float
        Specific density [-]
    nu : float
        Kinematic viscosity of water [m²/s]
    g : float, optional
        Gravitational acceleration [m/s²] (default = 9.81)

    Returns
    -------
    d_star : float
        Dimensionless particle diameter [-]
    """
    return d50 * ((Delta_s * g) / (nu**2)) ** (1.0 / 3.0)   


def bereken_armor_diameter(u_c, h, psi_c=0.03, K_s=1.0, K_v=1.0, delta=1.59):
    """
    Berekent de karakteristieke steendiameter d_n50 van een armorlaag (shields).

    Parameters
    ----------
    u_c : float
        Kritische, diepte-gemiddelde stroomsnelheid bij uniforme stroming [m/s]
    h: float
        Waterdiepte [m]
    psi_c : float, optional
        Drempel-van-beweging parameter (standaard 0.03)
    K_s : float, optional
        Sterkte-reductiefactor voor stenen op een talud [-]
    K_v : float, optional
        Snelheids-/turbulentiefactor [-]
    delta : float, optional
        Relatieve dichtheid (Δ = (ρ_s − ρ_w) / ρ_w) [-]

    Returns
    -------
    d : float
        Benodigde steendiameter [m]
    """
    C = 50.0 # Standaard Chezy-coëfficiënt
    
    for _ in range(10):
        d_n50 = (u_c**2) / (K_s * psi_c * delta * C**2)
        C = 18*np.log10(12*h/2*d_n50)
    d_n50 = (K_v ** 2 * u_c**2) / (K_s * psi_c * delta * C**2)

    return d_n50



In [ ]:
calculate_armor_diameter =  (u_c, C, psi_c=0.03, K_s=1.0, K_v=1.0, delta_C=1.0) => 